# Machine Learning Example With Cars 93 Dataset + Sklearn Learn Pipeline

In [104]:
#All imports included here
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from pandas_profiling import ProfileReport
import numpy as np
import os
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso ,RidgeCV,LassoCV , ElasticNet , ElasticNetCV, LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
os.chdir('/Users/rahulbanerjee/Rahul/ETL_HIVE/SQL')
#Import all pipeline dependency here
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [2]:
import pandas as pd
card_93_df = pd.read_csv('Cars93.csv')

# Starting with EDA

In [3]:
pf = ProfileReport(card_93_df)

In [4]:
pf.to_widgets()

# Define X and Y as the feature and label

In [105]:
y= card_93_df['Price']

In [106]:
x= card_93_df.drop(['Price','id','Min.Price'],axis=1)

# Pre-processing

In [107]:
#Find missing values and remove the same
#x.isna().sum()
for i in x.columns:
    if x[i].dtypes=='object':
        x[i]=x[i].fillna(x[i].mode()[0])
    else:
        x[i]=x[i].fillna(x[i].median())

In [108]:
#Breaking into cat and con vaiables
cat=[]
con=[]

for i in x.columns:
    if x[i].dtypes=='object':
        cat.append(i)
    else:
        con.append(i)

In [109]:
con_df = pd.DataFrame(x,columns=con)

# Looking at OLS model for Feature selection

In [110]:
from statsmodels.api import add_constant,OLS
ols=OLS(y,add_constant(con_df))

In [111]:
#For continous data only.
model=ols.fit()

In [112]:
#Take the first 10 values wrt to the P Value in ascending order
updated_col=model.pvalues.sort_values(ascending=True).index
updated_con_col = updated_col[0:12] 
updated_con_col = list(updated_con_col.drop('const'))

In [113]:
#Now we have the selected continous columns  + cat columns
selected_columns = updated_con_col + cat

In [136]:
#Now build the pipeline
num_pipe=Pipeline(steps=[('impute',SimpleImputer(strategy='median')),('scalar',StandardScaler())])
cat_pipe=Pipeline(steps=[('impute',SimpleImputer(strategy='most_frequent')),('Encode',OrdinalEncoder())])

In [137]:
#Now join the values
columnTransformer=ColumnTransformer([('num_pipe',num_pipe,updated_con_col),('cat_pipe',cat_pipe,cat)])
columnTransformer

ColumnTransformer(transformers=[('num_pipe',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='median')),
                                                 ('scalar', StandardScaler())]),
                                 ['Max.Price', 'Rear.seat.room', 'Passengers',
                                  'Length', 'EngineSize', 'Width', 'Horsepower',
                                  'Turn.circle', 'MPG.highway', 'Rev.per.mile',
                                  'Fuel.tank.capacity']),
                                ('cat_pipe',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encode', OrdinalEncoder())]),
                                 ['Manufacturer', 'Model', 'Type', 'AirBags',
                                  'DriveTrain', 'Cylinders', 'Man.trans.ava

In [138]:
#Now join the values and using fit transform().
selected_df=pd.DataFrame(columnTransformer.fit_transform(x),columns=selected_columns)

In [117]:
 selected_df

,Max.Price,Rear.seat.room,Passengers,Length,EngineSize,Width,Horsepower,Turn.circle,MPG.highway,Rev.per.mile,Fuel.tank.capacity,Manufacturer,Model,Type,AirBags,DriveTrain,Cylinders,Man.trans.avail,Origin,Make
0,-0.282465,-0.449727,-0.083243,-0.427186,-0.841022,-0.366184,-0.073484,-0.610436,0.360925,1.129530,-1.062184,0.0,48.0,3.0,2.0,1.0,1.0,1.0,1.0,0.0
1,1.531409,0.740403,-0.083243,0.812171,0.515869,0.431983,1.078322,-0.298510,-0.770514,0.005661,0.409445,0.0,55.0,2.0,0.0,1.0,3.0,1.0,1.0,1.0
2,0.948052,0.060329,-0.083243,-0.220626,0.128186,-0.632239,0.540813,-0.610436,-0.581941,-0.105713,0.072197,1.0,8.0,0.0,1.0,1.0,3.0,1.0,1.0,3.0
3,2.069191,1.080441,0.884457,0.674465,0.128186,0.165927,0.540813,-0.610436,-0.581941,0.410659,1.359872,1.0,0.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0
4,1.303535,-0.279708,-1.050944,0.192493,0.806631,-0.100128,1.231897,0.013416,0.172352,0.430909,1.359872,2.0,5.0,2.0,1.0,2.0,1.0,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.073018,2.100552,1.852158,0.261346,-0.162577,0.698038,-0.668585,-0.298510,-1.524806,1.180155,1.359872,30.0,41.0,5.0,2.0,1.0,2.0,1.0,1.0,88.0
89,0.045673,1.250459,-0.083243,-0.220626,-0.647181,-0.632239,-0.188665,-1.234288,0.172352,0.714407,0.562740,30.0,64.0,0.0,2.0,1.0,1.0,1.0,1.0,90.0
90,0.164167,-0.619745,-1.050944,-1.666543,0.128186,-0.898295,0.655993,-0.922362,-0.770514,0.106911,0.562740,30.0,28.0,4.0,2.0,1.0,3.0,1.0,1.0,87.0
91,0.145937,0.570385,-0.083243,0.467905,-0.356418,-0.632239,-0.572601,-0.610436,-0.204794,-0.237337,-0.265051,31.0,2.0,0.0,1.0,2.0,1.0,1.0,1.0,91.0


In [153]:
#Apply Train Test Split 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(selected_df,y,test_size=0.20,random_state=21)

In [154]:
#Model Building
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(x_train,y_train)

LinearRegression()

In [155]:
lm.score(x_test,y_test)

0.9753087395587918

In [156]:
#Trying to predict with one sample record ion the test .
single_test = pd.DataFrame(x_test.iloc[:1,:],columns=selected_columns)
#Predicted Value for x_train
y_pred_train=lm.predict(single_test)

In [122]:
y_pred_train

array([17.23177023])

In [98]:
#Now trying a randon value from the original data set x

In [157]:
#Transforming the original data set using transform only & not fit transform
x_sub_set_df =pd.DataFrame(columnTransformer.transform(x.iloc[0:99,:]),columns=selected_columns)

In [158]:
x_sub_set_y = lm.predict(x_sub_set_df)

In [159]:
x_sub_set_y

array([15.75552668, 34.84427802, 29.15413921, 37.71225336, 30.88077146,
       14.77199042, 20.59871034, 24.41515847, 24.56414963, 34.11925284,
       40.02453709, 14.63797587, 10.25078357, 16.24566207, 15.50766697,
       16.08550918, 16.54546289, 20.84057015, 38.10926443, 18.59916254,
       16.24869014, 27.74502367,  9.94071558, 12.63530302, 13.59483443,
       20.27954924, 14.86701903, 28.42777506, 13.43385348, 21.66153137,
        7.27005893, 10.04374084, 11.43361579, 17.42105288, 12.28877597,
       20.38886926, 22.97276486, 21.76625188,  7.83038286, 12.08500485,
       19.29565713, 12.5954729 , 19.08845325,  8.8333048 , 10.55085055,
       10.57774219, 13.57937858, 45.98692274, 26.19006984, 33.20882215,
       31.08734394, 34.97038765,  7.70233786, 10.36595609, 16.09244824,
       19.86392908, 32.0664052 , 28.08636522, 64.73412617, 12.17906299,
       16.23280098, 10.66324929, 27.90192075, 13.24981386, 15.98705968,
       19.25451761, 22.42171779, 13.28621775, 15.75449079, 18.09